In [1]:
!pip install simpletransformers pandas torch scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 876.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:0

In [2]:
import pandas as pd
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [5]:
train_df = pd.read_csv("/content/twitter_training.csv")
test_df = pd.read_csv("/content/twitter_test.csv")

In [6]:
print("\nTraining Data Preview:")
display(train_df.head())


Training Data Preview:


,Tweet ID,Entity,sentiment,Tweet Content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [7]:
print("\nTest Data Preview:")
display(test_df.head())


Test Data Preview:


,Tweet ID,Entity,sentiment,Tweet Content
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [8]:
train_df = train_df[['Tweet Content', 'sentiment']]
test_df = test_df[['Tweet Content', 'sentiment']]

In [9]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
train_df["sentiment"] = train_df["sentiment"].astype(str).str.strip()
test_df["sentiment"] = test_df["sentiment"].astype(str).str.strip()

In [10]:
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
train_df["sentiment"] = train_df["sentiment"].map(label_map)
test_df["sentiment"] = test_df["sentiment"].map(label_map)

In [11]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [12]:
train_df["sentiment"] = train_df["sentiment"].astype(int)
test_df["sentiment"] = test_df["sentiment"].astype(int)

In [13]:
print("\nTraining Dataset After Preprocessing:")
display(train_df.head())


Training Dataset After Preprocessing:


,Tweet Content,sentiment
0,im getting on borderlands and i will murder yo...,2
1,I am coming to the borders and I will kill you...,2
2,im getting on borderlands and i will kill you ...,2
3,im coming on borderlands and i will murder you...,2
4,im getting on borderlands 2 and i will murder ...,2


In [14]:
print("\nValidation Dataset After Preprocessing:")
display(test_df.head())


Validation Dataset After Preprocessing:


,Tweet Content,sentiment
1,BBC News - Amazon boss Jeff Bezos rejects clai...,1
2,@Microsoft Why do I pay for WORD when it funct...,0
3,"CSGO matchmaking is so full of closet hacking,...",0
4,Now the President is slapping Americans in the...,1
5,Hi @EAHelp I’ve had Madeleine McCann in my cel...,0


In [15]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 2
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.learning_rate = 2e-5
model_args.overwrite_output_dir = True
model_args.save_best_model = True
model_args.evaluate_during_training = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.output_dir = "mbert-sentiment-model"
model_args.best_model_dir = "mbert-best-model"

In [16]:
model = ClassificationModel(
    "bert", "bert-base-multilingual-cased",
    num_labels=3,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

print("\n👌 Model Loaded Successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]


👌 Model Loaded Successfully!


In [17]:
train_df, valid_df = train_test_split(train_df, train_size=0.8, shuffle=True, random_state=42)

In [18]:
print("\n🚀 Training Started...")
model.train_model(train_df, eval_df=valid_df)


🚀 Training Started...


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/97 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


...

  0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(6112,
 defaultdict(list,
             {'global_step': [2000, 3056, 4000, 6000, 6112],
              'train_loss': [0.5368900299072266,
               0.44737911224365234,
               0.3529701232910156,
               0.04889631271362305,
               0.33646249771118164],
              'mcc': [0.6377855173745354,
               0.7187506798547056,
               0.767748524211141,
               0.821550281062724,
               0.8220547684189525],
              'eval_loss': [0.6121235367519403,
               0.4831921066334045,
               0.4224530110951343,
               0.3387346484302695,
               0.33819837601356256]}))

In [19]:
print("\n🔍 Evaluating Model...")
result, model_outputs, wrong_predictions = model.eval_model(valid_df, acc=accuracy_score)



🔍 Evaluating Model...


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/765 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [20]:
print(f"\nModel Accuracy: {result['acc']:.4f}")


Model Accuracy: 0.8814


In [21]:
predictions, raw_outputs = model.predict(valid_df["Tweet Content"].tolist())

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/765 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [22]:
print("\n📊 Classification Report:\n")
print(classification_report(valid_df["sentiment"], predictions))


📊 Classification Report:

              precision    recall  f1-score   support

           0       0.90      0.90      0.90      4427
           1       0.91      0.83      0.87      3678
           2       0.85      0.90      0.87      4120

    accuracy                           0.88     12225
   macro avg       0.88      0.88      0.88     12225
weighted avg       0.88      0.88      0.88     12225

